In [1]:
from xgboost import XGBClassifier
from sklift.models import ClassTransformation
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k
from sklift.viz import plot_qini_curve
import matplotlib.pyplot as plt

import io

import pandas as pd
import numpy as np
from datetime import datetime
%matplotlib inline

C:\Anaconda\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Так как не было постоянного доступа к интернету, решила скачать файлы на компьютер.

In [2]:
df_clients = pd.read_csv("/kaggle/input/uplift-shift-23/x5-uplift-valid/data/clients2.csv",
                         index_col="client_id",
                         parse_dates=["first_issue_date", "first_redeem_date"])

df_products = pd.read_csv("/kaggle/input/uplift-shift-23/x5-uplift-valid/data/products.csv",
                          index_col="product_id")

df_train = pd.read_csv("/kaggle/input/uplift-shift-23/x5-uplift-valid/data/train.csv",
                       index_col="client_id")

df_test = pd.read_csv("/kaggle/input/uplift-shift-23/x5-uplift-valid/data/test.csv",
                      index_col="client_id")


Предварительный анализ

##### Клиенты

In [3]:
df_clients.head()

,client_id.1,first_issue_date,first_redeem_date,age,gender
client_id,,,,,
000012768d,NaN,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
000036f903,NaN,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
00010925a5,NaN,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U
0001f552b0,NaN,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F
00020e7b18,NaN,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U


In [4]:
df_clients["age"].describe()

count    200039.000000
mean         46.417329
std          49.532475
min       -7491.000000
25%          34.000000
50%          45.000000
75%          59.000000
max        1852.000000
Name: age, dtype: float64

In [5]:
df_clients["gender"].describe()

count     200039
unique         3
top            U
freq       92832
Name: gender, dtype: object

In [6]:
df_clients["first_issue_date"].describe()

C:\Users\user\AppData\Local\Temp\ipykernel_12208\3332607458.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_clients["first_issue_date"].describe()


count                  200039
unique                 199185
top       2017-07-12 13:53:06
freq                        3
first     2017-04-04 18:24:18
last      2019-03-15 21:44:14
Name: first_issue_date, dtype: object

In [7]:
df_clients["first_redeem_date"] = df_clients["first_redeem_date"].fillna(df_clients["first_issue_date"] - pd.DateOffset(1))

In [8]:
df_clients["first_redeem_date"].describe()

C:\Users\user\AppData\Local\Temp\ipykernel_12208\760325913.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_clients["first_redeem_date"].describe()


count                  200039
unique                 199428
top       2019-02-14 16:45:14
freq                        3
first     2017-04-09 09:31:36
last      2019-11-20 01:14:10
Name: first_redeem_date, dtype: object

#### Продукты

In [9]:
df_products.head()

,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
product_id,,,,,,,,,,
0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


In [10]:
# df_purchases получен путём слияния и сохранения двух csv, train_purch и test_purch
df_purchases = pd.read_csv("/kaggle/working/purchases.csv",
                           index_col="client_id",
                           parse_dates=["transaction_datetime"])

In [11]:
spenders = df_purchases.loc[df_purchases["express_points_spent"] != 0]

In [12]:
spenders

,Unnamed: 0,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
client_id,,,,,,,,,,,,,
00020e7b18,446,1281f2b1c4,2019-02-01 08:58:12,10.5,0.0,-3.0,-10.0,1070.85,2531f694dd,3e038662c0,1.0,57.0,58.0
00020e7b18,447,1281f2b1c4,2019-02-01 08:58:12,10.5,0.0,-3.0,-10.0,1070.85,2531f694dd,4009f09b04,1.0,5.0,5.0
00020e7b18,448,1281f2b1c4,2019-02-01 08:58:12,10.5,0.0,-3.0,-10.0,1070.85,2531f694dd,8dd55b4c39,2.0,48.0,48.0
00020e7b18,449,1281f2b1c4,2019-02-01 08:58:12,10.5,0.0,-3.0,-10.0,1070.85,2531f694dd,c455c81039,1.0,79.0,80.0
00020e7b18,450,1281f2b1c4,2019-02-01 08:58:12,10.5,0.0,-3.0,-10.0,1070.85,2531f694dd,b503272147,1.0,38.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff2e1fd45,22881802,884fcbfd9c,2019-03-09 09:07:51,1.7,0.0,-55.0,-30.0,425.48,d0ff8af489,68755eb373,2.0,144.0,180.0
fff2e1fd45,22881803,884fcbfd9c,2019-03-09 09:07:51,1.7,0.0,-55.0,-30.0,425.48,d0ff8af489,c32cffe924,1.0,50.0,63.0
fff2e1fd45,22881804,31f5b529ea,2019-03-16 11:47:10,0.5,0.0,-2.0,-30.0,149.93,d0ff8af489,0fbb27855f,1.0,12.0,15.0


In [14]:
df_purchases_agg = df_purchases.groupby(["client_id","transaction_id"])\
                   [["regular_points_received", "regular_points_spent",
                     "express_points_received", "express_points_spent",
                     "purchase_sum"]].last()\
                     .groupby("client_id").agg({
                         "regular_points_received":["mean", "sum"],
                         "regular_points_spent": ["mean", "sum"],
                         "express_points_received":["mean", "sum"],
                         "express_points_spent": ["mean", "sum"], 
                         "purchase_sum": ["sum"]
                         })

df_purchases_agg.columns = ["means_received", "regular_received_sum",
                            "regular_spent_mean", "regular_spent_sum",
                            "express_received_mean", "express_received_sum", 
                            "express_spent_mean", "express_spent_sum", "purchase_sums"]

df_purchases_agg = pd.read_csv("/kaggle/working/purchases_agg.csv",
                               index_col="client_id")
df_purchases_agg.head()

,regular_received_mean,regular_received_sum,regular_spent_mean,regular_spent_sum,express_received_mean,express_received_sum,express_spent_mean,express_spent_sum,purchase_sum_sum
client_id,,,,,,,,,
000012768d,6.425000,25.7,0.000000,0.0,0.000,0.0,0.000000,0.0,2803.00
000036f903,1.715625,54.9,0.000000,0.0,1.875,60.0,0.000000,0.0,9805.00
00010925a5,1.766667,31.8,-0.944444,-17.0,0.000,0.0,0.000000,0.0,5883.00
0001f552b0,5.260000,78.9,0.000000,0.0,0.000,0.0,0.000000,0.0,6155.18
00020e7b18,15.894444,286.1,-32.888889,-592.0,0.000,0.0,-1.666667,-30.0,25819.61


In [15]:
df_clients = df_clients.drop(df_clients[df_clients.age <= 7].index)
df_clients = df_clients.drop(df_clients[df_clients.age >= 91].index)

In [16]:
df_clients_together_purchase_agg = pd.merge(
    df_purchases_agg[
        ["express_spent_mean", "express_spent_sum", "purchase_sum_sum"]
    ],
    df_clients[
        ["first_redeem_date", "first_issue_date"]
    ], on="client_id", how="left")

df_clients_together_purchase_agg["differ_redeem"] = (df_clients_together_purchase_agg["first_redeem_date"] - df_clients_together_purchase_agg["first_issue_date"]) // pd.Timedelta("1d")

In [17]:
df_clients_together_purchase_agg.head()

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,first_issue_date,differ_redeem
client_id,,,,,,
000012768d,0.000000,0.0,2803.00,2018-01-04 19:30:07,2017-08-05 15:40:48,152.0
000036f903,0.000000,0.0,9805.00,2017-04-23 12:37:56,2017-04-10 13:54:23,12.0
00010925a5,0.000000,0.0,5883.00,2018-09-14 16:12:49,2018-07-24 16:21:29,51.0
0001f552b0,0.000000,0.0,6155.18,2018-08-28 12:59:45,2017-06-30 19:20:38,423.0
00020e7b18,-1.666667,-30.0,25819.61,2018-01-10 17:50:05,2017-11-27 11:41:45,44.0


In [18]:
df_clients_together_purchase_agg = df_clients_together_purchase_agg.groupby(["client_id"]).last()
df_clients_together_purchase_agg["redeem_after_treatment"] = (df_clients_together_purchase_agg["differ_redeem"] >= 0)
df_clients_together_purchase_agg = df_clients_together_purchase_agg.drop([ "first_issue_date"], axis=1)
df_clients_together_purchase_agg["first_redeem_date"] = ((df_clients_together_purchase_agg["first_redeem_date"] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1d"))
min_redeem = df_clients_together_purchase_agg["first_redeem_date"].min()
df_clients_together_purchase_agg["first_redeem_date"] = (df_clients_together_purchase_agg["first_redeem_date"] - min_redeem)

In [19]:
df_clients_together_purchase_agg.head()

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment
client_id,,,,,,
000012768d,0.000000,0.0,2803.00,270.0,152.0,True
000036f903,0.000000,0.0,9805.00,14.0,12.0,True
00010925a5,0.000000,0.0,5883.00,523.0,51.0,True
0001f552b0,0.000000,0.0,6155.18,506.0,423.0,True
00020e7b18,-1.666667,-30.0,25819.61,276.0,44.0,True


In [20]:
df_clients_together_purchase_agg = pd.merge(
    df_clients_together_purchase_agg,
    df_clients[["age"]], on="client_id", how="left"
)

df_clients_together_purchase_agg = pd.merge(df_clients_together_purchase_agg,pd.get_dummies(df_clients["gender"]),left_index=True, right_index=True)

df_clients_together_purchase_agg.head()

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment,age,F,M,U
client_id,,,,,,,,,,
000012768d,0.000000,0.0,2803.00,270.0,152.0,True,45.0,0,0,1
000036f903,0.000000,0.0,9805.00,14.0,12.0,True,72.0,1,0,0
00010925a5,0.000000,0.0,5883.00,523.0,51.0,True,83.0,0,0,1
0001f552b0,0.000000,0.0,6155.18,506.0,423.0,True,33.0,1,0,0
00020e7b18,-1.666667,-30.0,25819.61,276.0,44.0,True,73.0,0,0,1


In [21]:
df_cwp_train = pd.merge(
    df_clients_together_purchase_agg,
    df_train, on="client_id", how="left"
).dropna()

df_cwp_test = pd.merge(
    df_test, df_clients_together_purchase_agg,
    on="client_id", how="left"
)

In [22]:
df_cwp_test.shape

(60012, 10)

In [23]:
df_cwp_test.head()

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment,age,F,M,U
client_id,,,,,,,,,,
a9a604ed6e,0.0,0.0,3147.53,521.0,-1.0,False,36.0,1.0,0.0,0.0
ebd7360016,0.0,0.0,5680.57,145.0,59.0,True,63.0,1.0,0.0,0.0
908cd9b8e8,0.0,0.0,6193.04,460.0,105.0,True,49.0,1.0,0.0,0.0
dceb8ce861,0.0,0.0,8265.26,489.0,309.0,True,46.0,0.0,0.0,1.0
f4f0ac6b06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_final_train, df_final_test = train_test_split(df_cwp_train, random_state=42)
without_treatment = df_final_train.groupby("treatment_flg").count()["express_spent_mean"].loc[0]
with_treatment = df_final_train.groupby("treatment_flg").count()["express_spent_mean"].loc[1]

if with_treatment > without_treatment:
    df_final_train = df_final_train.drop(index=df_final_train[df_final_train["treatment_flg"] == 1].iloc[0:with_treatment - without_treatment].index)
elif with_treatment < without_treatment:
    df_final_train = df_final_train.drop(index=df_final_train[df_final_train["treatment_flg"] == 0].iloc[0:without_treatment - with_treatment].index)

without_treatment = df_final_train.groupby("treatment_flg").count()["express_spent_mean"].loc[0]
with_treatment = df_final_train.groupby("treatment_flg").count()["express_spent_mean"].loc[1]

without_treatment = df_final_test.groupby("treatment_flg").count()["express_spent_mean"].loc[0]
with_treatment = df_final_test.groupby("treatment_flg").count()["express_spent_mean"].loc[1]

if with_treatment > without_treatment:
    df_final_test = df_final_test.drop(index=df_final_test[df_final_test["treatment_flg"] == 1].iloc[0:with_treatment - without_treatment].index)
elif with_treatment < without_treatment:
    df_final_test = df_final_test.drop(index=df_final_test[df_final_test["treatment_flg"] == 0].iloc[0:without_treatment - with_treatment].index)

without_treatment = df_final_test.groupby("treatment_flg").count()["express_spent_mean"].loc[0]
with_treatment = df_final_test.groupby("treatment_flg").count()["express_spent_mean"].loc[1]

In [25]:
df_final_train.head()

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment,age,F,M,U,treatment_flg,purchases
client_id,,,,,,,,,,,,
f1ee23e6b9,0.0,0.0,6208.88,663.0,172.0,True,31.0,1,0,0,0.0,0.0
e342e244ea,0.0,0.0,1329.67,346.0,200.0,True,45.0,0,0,1,0.0,1.0
349d99be3d,0.0,0.0,9170.01,258.0,0.0,True,40.0,1,0,0,0.0,1.0
24dc9d4fe0,0.0,0.0,19079.00,444.0,3.0,True,45.0,0,0,1,0.0,1.0
1c17d8ff5b,0.0,0.0,762.00,721.0,67.0,True,36.0,1,0,0,0.0,0.0


In [26]:
df_final_test.head()

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment,age,F,M,U,treatment_flg,purchases
client_id,,,,,,,,,,,,
7af14c42e7,0.0,0.0,29840.82,569.0,298.0,True,39.0,0,0,1,0.0,0.0
688f951cdd,0.0,0.0,10950.03,660.0,242.0,True,52.0,0,0,1,0.0,1.0
049f7102f6,0.0,0.0,20457.57,467.0,119.0,True,36.0,0,1,0,0.0,0.0
9f2cb8e97a,0.0,0.0,4572.00,810.0,562.0,True,72.0,0,0,1,0.0,1.0
71e64a3d90,0.0,0.0,5709.08,426.0,195.0,True,37.0,0,0,1,0.0,1.0


In [27]:
df_final_test

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment,age,F,M,U,treatment_flg,purchases
client_id,,,,,,,,,,,,
7af14c42e7,0.0,0.0,29840.82,569.0,298.0,True,39.0,0,0,1,0.0,0.0
688f951cdd,0.0,0.0,10950.03,660.0,242.0,True,52.0,0,0,1,0.0,1.0
049f7102f6,0.0,0.0,20457.57,467.0,119.0,True,36.0,0,1,0,0.0,0.0
9f2cb8e97a,0.0,0.0,4572.00,810.0,562.0,True,72.0,0,0,1,0.0,1.0
71e64a3d90,0.0,0.0,5709.08,426.0,195.0,True,37.0,0,0,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3cd44897ed,0.0,0.0,4804.25,138.0,20.0,True,29.0,0,0,1,1.0,1.0
35b6882932,0.0,0.0,10504.00,492.0,88.0,True,46.0,0,0,1,1.0,1.0
044180d5e8,0.0,0.0,873.00,269.0,206.0,True,38.0,0,0,1,0.0,0.0


In [28]:
from sklift.metrics import qini_auc_score

qini_scores = []
max_depth= []
learning_rate = []
n_estimators = []

for depth in [1,2,3,4,5,6,7,8,9,10]:
    for lr in [0.01, 0.02, 0.05, 0.08, 0.1]:
        for n_est in [50, 100, 150, 200, 300]:
            xgb_params = {
                "max_depth":depth,
                "learning_rate": lr, 
                "n_estimators": n_est,
                "nthread":50,
                "seed":42,
                "use_label_encoder":False,
                "booster":"gbtree",
                "reg_alpha":0.9,
                "reg_lambda":0.9
            }

            estimator = XGBClassifier(
                    **xgb_params
                )
            uplift_model_cl_tr = ClassTransformation(
            estimator=estimator
            )

            uplift_model_cl_tr = uplift_model_cl_tr.fit(
                X=df_final_train.drop(
                    ["treatment_flg", "purchases"],
                    axis=1),
                    y=df_final_train["purchases"],
                    treatment=df_final_train["treatment_flg"])
            uplift_ts = uplift_model_cl_tr.predict(
                df_final_test.drop(["treatment_flg", "purchases"], axis=1),
            )
            qini_score = qini_auc_score(df_final_test["purchases"], uplift_ts, df_final_test["treatment_flg"])
            qini_scores.append(qini_score)
            max_depth.append(depth)            
            learning_rate.append(lr)            
            n_estimators.append(n_est)            
df = pd.DataFrame()
df["qini_score"] = qini_scores
df["max_depth"] = max_depth
df["learning_rate"] = learning_rate
df["n_estimators"]=n_estimators

C:\Anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[19:49:43] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[19:49:44] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


...

[20:35:34] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:35:55] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:36:20] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [29]:
df

,qini_score,max_depth,learning_rate,n_estimators
0,0.012406,1,0.01,50
1,0.021698,1,0.01,100
2,0.021954,1,0.01,150
3,0.022249,1,0.01,200
4,0.022442,1,0.01,300
...,...,...,...,...
245,0.018666,10,0.10,50
246,0.015996,10,0.10,100
247,0.014722,10,0.10,150
248,0.015732,10,0.10,200


In [30]:
maximums = df.max()
print(maximums)

qini_score         0.030808
max_depth         10.000000
learning_rate      0.100000
n_estimators     300.000000
dtype: float64


In [31]:
str = df[df["qini_score"]==df["qini_score"].max()]

In [32]:
str

,qini_score,max_depth,learning_rate,n_estimators
56,0.030808,3,0.02,100


In [33]:
xgb_params = {
    "max_depth":3,
    "learning_rate": 0.02, 
    "n_estimators": 100,
    "nthread":50,
    "seed":42,
    "use_label_encoder":False,
    "booster":"gbtree",
    "reg_alpha":0.9,
    "reg_lambda":0.9
}

uplift_model_cl_tr = ClassTransformation(estimator=XGBClassifier(**xgb_params)
)

uplift_model_cl_tr = uplift_model_cl_tr.fit(
    X=df_final_train.drop(
        ["treatment_flg", "purchases"],
        axis=1),
        y=df_final_train["purchases"],
        treatment=df_final_train["treatment_flg"])

df_cwp_test.fillna(0, inplace= True )
uplift_ts = uplift_model_cl_tr.predict(df_cwp_test)

C:\Anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[20:40:10] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [34]:
df_final_train = df_final_train.dropna()
df_final_train

,express_spent_mean,express_spent_sum,purchase_sum_sum,first_redeem_date,differ_redeem,redeem_after_treatment,age,F,M,U,treatment_flg,purchases
client_id,,,,,,,,,,,,
f1ee23e6b9,0.000000,0.0,6208.88,663.0,172.0,True,31.0,1,0,0,0.0,0.0
e342e244ea,0.000000,0.0,1329.67,346.0,200.0,True,45.0,0,0,1,0.0,1.0
349d99be3d,0.000000,0.0,9170.01,258.0,0.0,True,40.0,1,0,0,0.0,1.0
24dc9d4fe0,0.000000,0.0,19079.00,444.0,3.0,True,45.0,0,0,1,0.0,1.0
1c17d8ff5b,0.000000,0.0,762.00,721.0,67.0,True,36.0,1,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
ca9a01c5c4,0.000000,0.0,5816.00,432.0,417.0,True,63.0,1,0,0,1.0,1.0
dbf824b83e,0.000000,0.0,2963.00,265.0,110.0,True,59.0,0,0,1,1.0,0.0
be81507a91,0.000000,0.0,4322.25,295.0,72.0,True,41.0,0,0,1,1.0,1.0


In [35]:
len(uplift_ts)

60012

In [36]:
df_test["pred"] = uplift_ts

In [37]:
df_test = df_test.reset_index()
df_test

,client_id,pred
0,a9a604ed6e,0.021721
1,ebd7360016,0.028396
2,908cd9b8e8,0.021663
3,dceb8ce861,0.016155
4,f4f0ac6b06,0.017405
...,...,...
60007,4762fb04c5,0.018264
60008,533d1516e7,0.035287
60009,c93b21a707,-0.030739
60010,3fe0668de5,0.071493


In [38]:
df_test.to_csv(r"E:\job\shift\uplift-shift-23\x5-uplift-valid\data\result.csv", index=False)